In [2]:
import os

from getpass import getpass
import nest_asyncio

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()


CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OpenAI API key: ")
AZURE_AI_SEARCH_ENDPOINT= os.environ['AZURE_AI_SEARCH_ENDPOINT']
AZURE_AI_SEARCH_API_KEY= os.environ['AZURE_AI_SEARCH_API_KEY']
AZURE_OPENAI_ENDPOINT= os.environ['AZURE_OPENAI_ENDPOINT']
AZURE_API_KEY= os.environ['AZURE_API_KEY']

In [5]:
from azure.search.documents.indexes import SearchIndexClient
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore
from llama_index.vector_stores.azureaisearch import IndexManagement
from llama_index.core import StorageContext
from azure.core.credentials import AzureKeyCredential
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding


# Init and loading index from vector store

#### Init embed model (using my own embed model hosted on azure AI foundry)

In [6]:
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-3-small",
    deployment_name="text-embedding-3-small",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_API_KEY,
    api_version="2024-02-01", 
)

#### Init index_client 
`(using my own index (vector db - Azure AI search) hosted on azure AI foundry)`

In [8]:
credential = AzureKeyCredential(AZURE_AI_SEARCH_API_KEY)
index_client = SearchIndexClient(endpoint=AZURE_AI_SEARCH_ENDPOINT, credential=credential)

#### Init Vector Store

In [9]:
index_name = 'it_can_be_done'

vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    embed_model=embed_model,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
)

async_search_or_index_client is None. Depending on the client type passed in, sync or async functions may not work.


#### Providind vector_store as storage_context

In [10]:
storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
    )

#### Loading index from stored vectors

In [11]:
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, 
    embed_model=embed_model,
    storage_context=storage_context
)

<h5 style="color:green">Index Loaded successfully. [READY FOR QUERYING]</h5>